In [1]:
%matplotlib inline

import numpy as np
import torch
import scipy.io as sio
from functools import partial

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12

In [2]:
from scipy.integrate import tplquad 
from scipy.integrate import tplquad 

sph_lims = [[0, np.pi/2], [0, np.pi], [0, 2*np.pi]];

def sph2Quat(psi, theta, phi):
    return np.array([np.cos(psi),
                     np.sin(psi)*np.cos(theta), 
                     np.sin(psi)*np.sin(theta)*np.cos(phi), 
                     np.sin(psi)*np.sin(theta)*np.sin(phi)])

def sph_dV(psi, theta, phi):
    return np.sin(psi)**2*np.sin(theta)

def integrate3Sphere(func):
    def sph_func_int(psi, theta, phi, ): 
        return np.asscalar(func(sph2Quat(psi, theta, phi))*sph_dV(psi, theta, phi))

    return tplquad(sph_func_int, 
                   sph_lims[0][0], sph_lims[0][1], 
                   sph_lims[1][0], sph_lims[1][1], 
                   sph_lims[2][0], sph_lims[2][1])    

In [3]:
from scipy.integrate import tplquad 

const_func = lambda q: 1.0
res = integrate3Sphere(const_func)
print('{} +/- {}'.format(res[0] - np.pi**2, res[1]))

0.0 +/- 2.3058791671639882e-09


In [4]:
from object_pose_utils.utils.interpolation import GaussianInterpolation, BinghamInterpolation
from object_pose_utils.utils import to_np

q_center = np.array([0.5377, 1.8339, -2.2588, 0.8622])
q_center /= np.linalg.norm(q_center)

w = [1]

sigma_gauss = np.pi/9
#sigma_bingham = sigma_gauss*2.8474391664672476
sigma_bingham = sigma_gauss*100

bingham_interp = BinghamInterpolation(vertices = [q_center], 
                                      values = torch.Tensor(w), 
                                      sigma=sigma_bingham)
gaussian_interp = GaussianInterpolation(vertices = [q_center], 
                                        values = w, 
                                        sigma=sigma_gauss)

if False:
    gauss_interp_func = lambda q: to_np(gaussian_interp(torch.Tensor(q).unsqueeze(0).cuda()))[0]
    bingham_interp_func = lambda q: to_np(bingham_interp(torch.Tensor(q).unsqueeze(0).cuda()))[0]
    #print('Gaussian: ', integrate3Sphere(gauss_interp_func))
    print('Bingham: ', integrate3Sphere(bingham_interp_func))

if False:
    N = 100
    q = np.random.randn(4,N)
    q = (q / np.linalg.norm(q, axis=0)).T
    
    p_gauss = to_np(gaussian_interp(torch.Tensor(q).cuda()))
    p_bingham = to_np(bingham_interp(torch.Tensor(q).cuda()))
    
    plt.plot(p_gauss, label = 'Gaussian')
    plt.plot(p_bingham, label = 'Bingham')
    plt.legend()

In [24]:
def logSumExp(x, weights, eta):
    max_x = torch.max(x, dim=1)[0]
    l_x = max_x + torch.log((weights*torch.exp(x - max_x.unsqueeze(1))).sum(1)) - np.log(eta)
    return l_x

In [27]:
import quat_math
#print(to_np(gaussian_interp(torch.Tensor([1,0,0,0]).unsqueeze(0).cuda())))
#print(to_np(gaussian_interp(torch.Tensor(q_center).unsqueeze(0).cuda())))
#print(gaussian_interp.eta)
#print(np.mean(p_gauss))
bingham_interp_twin = BinghamInterpolation(vertices = [q_center, 
                                                       np.array([1,0,0,0]), 
                                                       np.array([0,1,0,0])], 
                                           values = torch.Tensor([1, 1, 1]),
                                           sigma=sigma_bingham)

v = to_np(bingham_interp(torch.Tensor([1,0,0,0]).unsqueeze(0).cuda()))
print(np.log(v)[0])
v_exp = bingham_interp(torch.Tensor([1,0,0,0]).unsqueeze(0).cuda(), True)
print(logSumExp(v_exp, bingham_interp.values, bingham_interp.eta))
print('-'*10)

N = 100
q = np.random.randn(4,N)
q = (q / np.linalg.norm(q, axis=0)).T

q_test = quat_math.random_quaternion()
v = to_np(bingham_interp_twin(torch.Tensor(q).cuda()))
#print(v)
print(np.sum(np.log(v)))
v_exp = bingham_interp_twin(torch.Tensor(q).cuda(), True)
print(logSumExp(v_exp, bingham_interp_twin.values.repeat(q.shape[0],1), bingham_interp_twin.eta).sum())

likelihoods = []
for j in range(10):
    q_test = np.random.randn(4,1)
    q_test = (q_test / np.linalg.norm(q_test, axis=0)).T
    likelihoods.append((bingham_interp_twin(torch.Tensor(q_test).cuda(), True), bingham_interp_twin.values))
    

-30.254585
tensor([-30.2546], device='cuda:0')
----------
-1420.3512
tensor(-1420.3513, device='cuda:0')


In [56]:
l_exp, w = zip(*likelihoods)
l_exp = torch.cat(l_exp)
w = torch.stack(w)
print(l_exp)
print(w)
logSumExp(l_exp, w, bingham_interp_twin.eta)

tensor([[-34.7398, -34.6260, -30.5892],
        [-34.0658, -32.6413, -22.6942],
        [-23.2643, -28.2555,  -9.6411],
        [-34.8407, -34.4944, -18.9099],
        [-34.7151, -29.0957, -30.6557],
        [-11.9311, -32.9457, -16.7423],
        [-32.5241, -33.8145, -34.7993],
        [-34.1637, -26.9785, -30.2884],
        [-21.0640, -32.0214,  -3.7636],
        [-27.9597, -33.8896, -32.8215]], device='cuda:0')
tensor([[0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333]], device='cuda:0')


tensor([-28.0656, -20.2034,  -7.1503, -16.4191, -26.4112,  -9.4322, -29.7127,
        -24.4511,  -1.2728, -25.4585], device='cuda:0')

In [23]:
x = v_exp
weights = bingham_interp_twin.values.repeat(q.shape[0],1)
max_x = torch.max(x, dim=1)[0]
print(weights.shape)
print(x.shape)
print(max_x.shape)
print(torch.exp(x - max_x.unsqueeze(1)).shape)
max_x + (weights * torch.exp(x - max_x.unsqueeze(1))).sum(1)

torch.Size([100, 3])
torch.Size([100, 3])
torch.Size([100])
torch.Size([100, 3])


tensor([ -5.3842,  -9.6909, -10.6976,  -5.8070, -26.6386, -19.0015, -10.6848,
        -31.6912,  -6.3704, -11.1440, -21.1845, -14.6112,  -0.1661, -14.6903,
        -21.8387,  -2.6652, -13.0633, -10.4276, -20.7448,  -9.9265, -24.2304,
        -29.9733, -26.5406, -32.6465, -26.2746,  -5.5040, -13.9149,  -5.3910,
        -30.0625, -11.1359, -15.0516, -13.1476,  -5.8806,  -8.3395, -17.0903,
        -11.4056, -22.5381, -28.6831, -26.5702, -20.7181, -28.5862, -24.6759,
        -10.5092, -18.0649,  -8.7617, -11.1722, -17.5499,  -5.7197, -19.7160,
         -9.0251, -10.9634, -26.9137, -16.6863, -32.2655, -23.8396, -13.7072,
        -29.3784, -19.1985, -15.5046,  -8.5452,  -8.4332, -14.0292, -31.5896,
        -18.2666, -12.7678, -25.5955,  -2.4624,  -7.3991, -17.6156, -20.8727,
        -19.6057,  -3.7688, -17.3893, -21.9828, -33.2371,  -6.3811, -23.7905,
        -14.1455, -15.3075, -12.2345,  -3.9880, -26.1087, -27.2890, -23.7684,
         -9.9968, -13.4618, -19.7251, -18.5471, -20.1597, -16.69

In [ ]:
from quat_math import quat2AxisAngle 

N = 100000
q = np.random.randn(4,N)
q = (q / np.linalg.norm(q, axis=0)).T

p_bingham = to_np(bingham_interp(torch.Tensor(q).cuda())).flatten()
num_samples = 100
sample_idxs = np.random.choice(np.arange(N), num_samples, p=p_bingham/np.sum(p_bingham))
p_samples = p_bingham[sample_idxs]
q_samples = q[sample_idxs]
aa_samples = []
for j in sample_idxs:
    xi, theta = quat2AxisAngle(q[j])
    aa_samples.append(xi*theta)

aa_samples = np.array(aa_samples)

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(aa_samples[:,0], aa_samples[:,1], aa_samples[:,2], c=p_samples)
ax.axis('equal')
ax.set_xlim3d(-np.pi, np.pi)
ax.set_ylim3d(-np.pi, np.pi)
ax.set_zlim3d(-np.pi, np.pi)
plt.show()

In [ ]:
S = q_samples.T.dot(q_samples)/num_samples
e_vals, e_vecs = np.linalg.eig(S)
V = e_vecs[:, :3]
dY = e_vals[:-1]


In [ ]:
print(Z_hat)
print(Z)
print(M_hat)
print(M)

In [ ]:
from object_pose_utils.utils.grid_interpolation import gramSchmidt
M = gramSchmidt(q_center)
Z = np.diag([0,-sigma_bingham, -sigma_bingham, -sigma_bingham])
eta = bingham_interp.eta
bingham_true = lambda v: 1/eta * np.exp(v.dot(M).dot(Z).dot(M.T).dot(v.T))
print(bingham_true(np.array([1,0,0,0])))
print(bingham_true(q_center))
print('-'*10)
bingham_est = lambda v: np.exp(v.dot(M_hat).dot(Z_hat).dot(M_hat.T).dot(v.T))
print(bingham_est(np.array([1,0,0,0])))
print(bingham_est(q_center))
print('='*10)
print(bingham_est(np.array([1,0,0,0])) / bingham_true(np.array([1,0,0,0])))
print(bingham_est(np.array([0,1,0,0])) / bingham_true(np.array([0,1,0,0])))
print(bingham_est(np.array([0,0,1,0])) / bingham_true(np.array([0,0,1,0])))
print(bingham_est(np.array([0,0,0,1])) / bingham_true(np.array([0,0,0,1])))
print(bingham_est(q_center) / bingham_true(q_center))


In [ ]:
from object_pose_utils.utils.subrandom import subrandom
N = 20
srand = subrandom(N, dim=2)
rand = np.random.rand(N,2)

plt.subplot(121)
plt.scatter(srand[:,0], srand[:,1], s=5e2/N)
plt.title('Subrandom {}'.format(N))
plt.subplot(122)
plt.scatter(rand[:,0], rand[:,1], s=5e2/N)
plt.title('Random {}'.format(N))
plt.show()


srand = subrandom(N)
rand = np.random.rand(N)
plt.title('1-Dimensional {}'.format(N))
plt.scatter(srand[:], 0*srand[:], marker = '+', label='subrandom')
plt.scatter(rand[:], 0*rand[:]+1, marker = 'x', label='random')
plt.legend()
plt.show()